<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.utils.data
from torch import nn
from torchvision import models
import sys, os
# print(os.getcwd())
sys.path.append('./src')
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import patchcore
from patchcore import patchcore as patchcore_model
from siamese_full import SiameseNetwork
from torch.utils.data import TensorDataset, ConcatDataset, DataLoader
from torchvision import transforms
from typing import Callable, Any
import glob, os
import random
import numpy as np

class Custom_Dataset_All(Dataset):
    """
    URL = https://www.robots.ox.ac.uk/~vgg/data/pets/
    """
    def __init__(self,
                 image_folder: str = r"xx\mvtech_anomaly_detection\mvtech_anomaly_detection",
                 image_name: str = "bottle",
                 split: str = "train",
                 transform: Callable = None,
                **kwargs):
        self._sub_dataset = ('bottle',  'cable',  'capsule',  'carpet',  'grid', 'hazelnut', 'leather',  'metal_nut',  'pill', 'screw', 'tile', 'toothbrush', 'transistor', 'wood', 'zipper')
        self.image_name = image_name
        self.img_folder = image_folder
        self.transforms = transform
        self.imgs_pos, self.imgs_neg = self.get_image_paths(img_folder=image_folder)

    def get_image_paths(self, img_folder, split: str = "train"):
        img_positive = []
        img_negative = []
        # Define the file extensions for images
        image_extensions = ('*.png', '*.jpg', '*.jpeg', '*.gif', '*.bmp', '*.tiff')
        # Initialize an empty list to store image paths
        for img_name in self._sub_dataset:
            for ext in image_extensions:
                img_fold = os.path.join(img_folder, img_name, "train", "good")
                if (img_name == self.image_name):
                    img_positive.extend(glob.glob(os.path.join(img_fold, ext)))
                else:
                    img_negative.extend(glob.glob(os.path.join(img_fold, ext)))

        img_positive = img_positive[:int(len(img_positive) * 0.75)] if split == "train" else img_positive[int(len(img_positive) * 0.75):]
        img_positive = [self.pil_loader(x) for x in img_positive]
        img_negative = [self.pil_loader(x) for x in img_negative]
        random.shuffle(img_positive)
        random.shuffle(img_negative)
        return img_positive, img_negative

    def __len__(self):
        return len(self.imgs_pos)

    def pil_loader(self, path: str) -> Image.Image:
        # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
        # with open(path, "r") as f:
        img = Image.open(path).convert("RGB")
        return img

    def default_loader(self, path: str) -> Any:
            return self.pil_loader(path)

    def __getitem__(self, idx):
        valid_indices = [i for i in range(len(self.imgs_pos)) if i != idx]
        random_index = random.choice(valid_indices)
        img_anchor = self.imgs_pos[idx]
        img_pos = self.imgs_pos[random_index]
        img_neg = self.imgs_neg[random.randint(0, len(self.imgs_neg) - 1)]
        if self.transforms is not None:
            img_anchor = self.transforms(img_anchor)
            img_pos = self.transforms(img_pos)
            img_neg = self.transforms(img_neg)

        return img_anchor, img_pos,  img_neg

def _standard_patchcore(image_dimension):
    patchcore_instance = patchcore_model.PatchCore(torch.device("cpu"))
    backbone = models.wide_resnet50_2(pretrained=False)
    backbone.name, backbone.seed = "wideresnet50", 0
    patchcore_instance.load(
        backbone=backbone,
        layers_to_extract_from=["layer2", "layer3"],
        device=torch.device("cpu"),
        input_shape=[3, image_dimension, image_dimension],
        pretrain_embed_dimension=1024,
        target_embed_dimension=1024,
        patchsize=3,
        patchstride=1,
        spade_nn=2,
        sub_model=None
    )
    return patchcore_instance

if __name__ == "__main__":
    epoch_num = 200
    batch_size = 8
    IMAGENET_MEAN = [0.485, 0.456, 0.406]
    IMAGENET_STD = [0.229, 0.224, 0.225]
    network = SiameseNetwork(in_dimension=3).to(device='cuda')
    criterion = nn.TripletMarginLoss()
    criterion_2 = nn.MSELoss()
    optimizer = torch.optim.Adam(network.parameters(), lr=0.001)
    transfomation = transforms.Compose([
            transforms.Resize(256),
            # transforms.CenterCrop(224),
            # transforms.Resize(256),
            transforms.ToTensor(),
            # transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
        ])
    _dataset = Custom_Dataset_All(transform=transfomation)
    _data_loader = DataLoader(_dataset, batch_size=batch_size, shuffle=True)

    _dataset_test = Custom_Dataset_All(transform=transfomation, split="test")
    _test_data_loader = DataLoader(_dataset_test, batch_size=batch_size, shuffle=True)
    mean, std = 0, 0.2
    # model = _standard_patchcore(224)
    for epoch in range(epoch_num):
        print(f"=========== epoch {epoch} ===========")
        loss_item = 0.
        valid_loss = 0.
        network.train()
        for i, image in enumerate(_data_loader):
            if (image[0].shape[0] < batch_size): continue
            # image = torch.concat(image, dim=0)
            # feature = model.get_feature(image)
            bs, c, h, w = image[0].shape
            # gaussian_noise = torch.randn(1, c, h, w) * std + mean + image[0]
            img0, img1 = image[0], image[1]
            img0, img1 = img0.cuda(), img1.cuda()

            optimizer.zero_grad()
            output1, output2, output3, out_noise = network(img0, img1)
            loss = criterion(output1, output2, output3) * 1000 + criterion_2(img0, out_noise)  * 0.5
            loss.backward()
            optimizer.step()
            loss_item += loss.item()
            if i % 1 == 0:
                print(f"Epoch [{epoch+1}/{epoch_num}], Step [{i}/{len(_data_loader)}], Loss: {(loss_item/(i+1)):.4f}")


        network.eval()
        with torch.no_grad():
            for i, image in enumerate(_test_data_loader):
                if (image[0].shape[0] < 2): continue
                img0, img1 = image[0], image[1]
                img0, img1 = img0.cuda(), img1.cuda()

                output1, output2, output3, out_noise = network(img0, img1)
                loss = criterion(output1, output2, output3) * 1000 + criterion_2(img0, out_noise) * 0.5
                valid_loss += loss.item()
            print(f"Epoch [{epoch+1}/{epoch_num}], Valid Loss: {(valid_loss/len(_test_data_loader)):.4f}")


    torch.save(network.state_dict(), f'./sub_model/siamese_wide_resnet50_2_final.pth')

In [ ]:
import torch
from torch import nn

from torchsummary import summary

class GaussianNoise(nn.Module):
    """Gaussian noise regularizer.

    Args:
        sigma (float, optional): relative standard deviation used to generate the
            noise. Relative means that it will be multiplied by the magnitude of
            the value your are adding the noise to. This means that sigma can be
            the same regardless of the scale of the vector.
        is_relative_detach (bool, optional): whether to detach the variable before
            computing the scale of the noise. If `False` then the scale of the noise
            won't be seen as a constant but something to optimize: this will bias the
            network to generate vectors with smaller values.
    """
    def __init__(self, sigma=0.1, is_relative_detach=True, device='cuda'):
        super().__init__()
        # self.sigma = sigma
        self.sigma = nn.Parameter(torch.randn(1).to(device=device))
        self.is_relative_detach = is_relative_detach
        self.register_buffer('noise', torch.tensor(0).to(device=device))

        # self.noise = torch.tensor(0).to(device)

    def forward(self, x):
        if self.sigma != 0:
        # if self.training != 0:
            scale = self.sigma * x.detach() if self.is_relative_detach else self.sigma * x
            sampled_noise = self.noise.expand(*x.size()).float().normal_() * scale
            x = x + sampled_noise
        return x

class SiameseNetwork(nn.Module):
    def __init__(self, in_dimension=3, out_dimension=1024, device='cuda'):
        super(SiameseNetwork, self).__init__()

        self.out_dimension = out_dimension
        self.in_dimension = in_dimension
        self.device = device

        self.fcn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            # nn.Conv2d(32, 64, 3, 2),
            # nn.BatchNorm2d(64),
            # nn.ReLU(),

            nn.Conv2d(64, 128, 3, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            # nn.Conv2d(128, 256, 3, 2),
            # nn.BatchNorm2d(256),
            # nn.ReLU(),

            nn.Conv2d(128, 512, 3, 2, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.Conv2d(512, 1024, 3, padding='same'),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
        )
        self.nn_noise = GaussianNoise(is_relative_detach=False, device=device)
        # self.mean = nn.Parameter(torch.randn(1).to(device=device))
        # self.std = nn.Parameter(torch.randn(1).to(device=device))
        self.linear_sub_1 = nn.LazyLinear(out_features=256)
        self.linear_sub_2 = nn.LazyLinear(out_features=1024)
        self.global_pooling = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()


    def forward_once(self, x, add_noise=False):
    # def forward(self, x):
        # Forward pass
        if add_noise:
            # bs, c, w, h = x.shape
            # gaussian_noise = torch.randn(1, c, w, h).to(device=self.device) * self.std + self.mean
            # x = x + gaussian_noise
            out_noise = self.nn_noise(x)

        x = self.fcn(x)

        global_pooling = self.global_pooling(x)

        attention_score = torch.transpose(global_pooling, -3, -1)
        attention_score = self.linear_sub_1(attention_score)
        attention_score = self.relu(attention_score)
        attention_score = self.linear_sub_2(attention_score)
        attention_score = self.sigmoid(attention_score)
        attention_score = torch.transpose(attention_score, -3, -1)

        output = torch.mul(x, attention_score)
        if add_noise:
            return output, out_noise
        # output = torch.reshape(output, (-1, self.out_dimension))
        return output

    # def forward(self, input1, input2):
    #     # forward pass of input 1
    #     output1 = self.forward_once(torch.unsqueeze(input1[0,:,:,:], dim=0), torch.unsqueeze(input2[0,:,:,:], dim=0))
    #     # forward pass of input 2
    #     output2 = self.forward_once(torch.unsqueeze(input1[1,:,:,:], dim=0), torch.unsqueeze(input2[1,:,:,:], dim=0))
    #     return output1, output2

    def forward(self, input1, input2):
        # forward pass of input 1
        output1, output_noise = self.forward_once(input1, add_noise=True)
        # forward pass of input 2
        output2 = self.forward_once(input2)
        # forward pass of input 2
        output3 = self.forward_once(output_noise)
        return output1, output2, output3, output_noise


# network = SiameseNetwork(3, 1024).to(device='cuda')
# summary(network, ((3, 224, 224)))
# tensor = torch.randn(4, 3, 224, 224).to(device='cuda')
# print(network(tensor).shape)